In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
#import dlib
import os
import pickle
np.random.seed(1000)


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
frames = []
labels = []
for file in os.listdir('output/'):
    if file[-10:] == 'frames.pkl':
        with open('output/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        with open('output/'+file, 'rb') as f:
            labels.append(pickle.load(f))

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [label[0]] * len(clip)
    except:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [label[0]] * len(clip)
    except:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [7]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for i in range(len(train_images)):
    train_images[i] = clahe.apply(train_images[i])
for i in range(len(test_images)):
    test_images[i] = clahe.apply(test_images[i])
   

In [8]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [9]:
test_images, test_labels =np.asarray(test_images), np.asarray(test_labels)

In [10]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [11]:
train_labels //= 2
test_labels //= 2

In [12]:
#print((train_labels == 0).sum())
#(train_labels == 1).sum()


In [13]:
import tensorflow.keras.layers as kl
import tensorflow.keras.losses
from tensorflow.keras.applications.resnet50 import ResNet50

In [14]:
def network():
    model = tf.keras.Sequential()
    model.add(kl.InputLayer(input_shape=(48, 48, 1)))
    # First conv block
    model.add(kl.Conv2D(filters = 96, kernel_size=7, padding='same', strides=2))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Second conv block
    model.add(kl.Conv2D(filters = 144, kernel_size=5, padding='same', strides=1))
    model.add(tf.keras.layers.ReLU())
    model.add(kl.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.5))
    # Third-Fourth-Fifth conv block
    for i in range(3):
        model.add(kl.Conv2D(filters = 144, kernel_size=3, padding='same', strides=1))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(kl.MaxPooling2D(pool_size=(3, 3)))
    # Flatten
    model.add(kl.Flatten())
    # First FC 
    model.add(kl.Dense(4048))
    # Second Fc
    model.add(kl.Dense(4048))
    # Third FC
    model.add(kl.Dense(4))
    # Softmax at the end
    model.add(kl.Softmax())
    
    return model

In [15]:
model = network()
InitialLearnRate = 0.03
MaxEpochs = 30
MiniBatchSize = 32
opt = tf.keras.optimizers.SGD(lr=InitialLearnRate, decay=InitialLearnRate / MaxEpochs)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])


In [16]:
#model.compile(optimizer='adamax',
 #            loss='sparse_categorical_crossentropy',
  #            metrics=['accuracy'])

In [17]:
from sklearn.utils.class_weight import compute_class_weight 
weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
print(weights)

[0.55091863 5.40979479]


In [18]:
weights = {0:0.68708684, 1:1.83627784}
print(weights)

{0: 0.68708684, 1: 1.83627784}


In [19]:
model.fit(train_images, train_labels, epochs=3, class_weight=weights, batch_size=50)

  ...
    to  
  ['...']
Train on 1944475 samples
Epoch 1/3
1944475/1944475 [==============================] - 267s 137us/sample - loss: nan - accuracy: 0.9075
Epoch 2/3
1944475/1944475 [==============================] - 263s 135us/sample - loss: nan - accuracy: 0.9076
Epoch 3/3
1944475/1944475 [==============================] - 264s 136us/sample - loss: nan - accuracy: 0.9076


In [20]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

482212/482212 [==============================] - 33s 68us/sample - loss: nan - accuracy: 0.8955


In [22]:
model.save('confusion.h5')
print("finish")

finish
